# Estimate Binomial draw probabilility

In [1]:
using StatisticalRethinking
using DynamicHMC, TransformVariables, LogDensityProblems, MCMCDiagnostics
using Parameters, ForwardDiff

Define a structure to hold the data.

In [2]:
struct BernoulliProblem
    "Total number of draws in the data."
    n::Int
    "Number of draws `==1` in the data"
    s::Vector{Int}
end;

Make the type callable with the parameters *as a single argument*.

In [3]:
function (problem::BernoulliProblem)((α, )::NamedTuple{(:α, )})
    @unpack n, s = problem        # extract the data
    loglikelihood(Binomial(n, α), s)
end

Create the data and complete setting up the problem.

In [4]:
obs = rand(Binomial(9, 2/3), 1)
p = BernoulliProblem(9, obs)
p((α = 0.5, ))

-2.6548056865833978

Use a flat priors (the default, omitted) for α

In [5]:
P = TransformedLogDensity(as((α = as𝕀,)), p)
∇P = ADgradient(:ForwardDiff, P);

Sample

In [6]:
chain, NUTS_tuned = NUTS_init_tune_mcmc(∇P, 1000)

MCMC, adapting ϵ (75 steps)
0.0011 s/step ...done
MCMC, adapting ϵ (25 steps)
6.8e-6 s/step ...done
MCMC, adapting ϵ (50 steps)
0.0027 s/step ...done
MCMC, adapting ϵ (100 steps)
1.3e-5 s/step ...done
MCMC, adapting ϵ (200 steps)
7.1e-6 s/step ...done
MCMC, adapting ϵ (400 steps)
1.2e-5 s/step ...done
MCMC, adapting ϵ (50 steps)
1.5e-5 s/step ...done
MCMC (1000 steps)
7.3e-6 s/step ...done


(DynamicHMC.NUTS_Transition{Array{Float64,1},Float64}[NUTS_Transition{Array{Float64,1},Float64}([1.00657], -3.38966, 2, DoubledTurn, 1.0, 3), NUTS_Transition{Array{Float64,1},Float64}([0.900219], -2.86914, 2, AdjacentTurn, 0.999272, 7), NUTS_Transition{Array{Float64,1},Float64}([0.160142], -3.84618, 1, AdjacentTurn, 0.851731, 3), NUTS_Transition{Array{Float64,1},Float64}([0.760105], -3.56315, 2, AdjacentTurn, 1.0, 7), NUTS_Transition{Array{Float64,1},Float64}([1.62986], -3.28392, 2, AdjacentTurn, 0.950921, 7), NUTS_Transition{Array{Float64,1},Float64}([1.74206], -3.44488, 1, DoubledTurn, 0.980677, 1), NUTS_Transition{Array{Float64,1},Float64}([1.35462], -4.34427, 2, DoubledTurn, 0.928081, 3), NUTS_Transition{Array{Float64,1},Float64}([0.7817], -2.99616, 2, AdjacentTurn, 0.99922, 7), NUTS_Transition{Array{Float64,1},Float64}([1.07517], -3.76806, 2, DoubledTurn, 0.88463, 3), NUTS_Transition{Array{Float64,1},Float64}([1.12476], -2.88413, 1, DoubledTurn, 0.997588, 1)  …  NUTS_Transition{Ar

To get the posterior for ``α`` use `get_position` and then transform back.

In [7]:
posterior = TransformVariables.transform.(Ref(∇P.transformation), get_position.(chain));

Extract the parameter.

In [8]:
posterior_α = first.(posterior);

check the effective sample size

In [9]:
ess_α = effective_sample_size(posterior_α)

360.9327929049132

NUTS-specific statistics

In [10]:
NUTS_statistics(chain)

Hamiltonian Monte Carlo sample of length 1000
  acceptance rate mean: 0.94, min/25%/median/75%/max: 0.33 0.92 0.98 1.0 1.0
  termination: AdjacentTurn => 43% DoubledTurn => 57%
  depth: 1 => 57% 2 => 40% 3 => 3%


check the mean

In [11]:
mean(posterior_α)

0.7231894482788868

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*